# Model Selection for SLP Prediction (Multi-Dataset)

This notebook performs model selection to predict the `slp` column using various machine learning algorithms with time series cross-validation.

**Processes 3 datasets:**
- `df_clean_stat.csv` → `data_v3_stat.csv` (statistical features)
- `df_clean_rf.csv` → `data_v3_rf.csv` (random forest features)
- `data_v2_full.csv` → `data_v3_full.csv` (all features with cyclical encoding)


In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import clone
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

# Define all datasets to process
DATASETS = [
    {
        'name': 'stat',
        'input_file': 'dataset/df_clean_stat.csv',
        'output_file': 'dataset/data_v3_stat.csv',
        'target_col': 'Residential (SLP)',
        'has_cyclical_cols': False,
        'drop_cols': ['rlm', 'entry'],
    },
    {
        'name': 'rf',
        'input_file': 'dataset/df_clean_rf.csv',
        'output_file': 'dataset/data_v3_rf.csv',
        'target_col': 'slp',
        'has_cyclical_cols': False,
        'drop_cols': ['idx'],
    },
    {
        'name': 'full',
        'input_file': 'dataset/data_v2_full.csv',
        'output_file': 'dataset/data_v3_full.csv',
        'target_col': 'slp',
        'has_cyclical_cols': True,
        'drop_cols': ['rlm', 'entry'],
    },
]

print(f"Configured {len(DATASETS)} datasets to process:")
for ds in DATASETS:
    print(f"  - {ds['name']}: {ds['input_file']} → {ds['output_file']}")


Configured 3 datasets to process:
  - stat: dataset/df_clean_stat.csv → dataset/data_v3_stat.csv
  - rf: dataset/df_clean_rf.csv → dataset/data_v3_rf.csv
  - full: dataset/data_v2_full.csv → dataset/data_v3_full.csv


## 1. Process All Datasets

This cell loops through all 3 datasets, performing:
1. Data loading and cleaning
2. Feature preprocessing (cyclical encoding, scaling)
3. Model training and evaluation with TimeSeriesSplit
4. Optimal training timespan analysis
5. Export of processed data


In [29]:
# Store results for all datasets
all_results = {}
all_timespan_results = {}

# Process each dataset
for dataset_config in DATASETS:
    dataset_name = dataset_config['name']
    dataset_file = dataset_config['input_file']
    output_file = dataset_config['output_file']
    target_col = dataset_config['target_col']
    has_cyclical = dataset_config['has_cyclical_cols']
    drop_cols = dataset_config['drop_cols']
    
    print("\n" + "=" * 80)
    print(f"PROCESSING DATASET: {dataset_name.upper()}")
    print(f"Input: {dataset_file}")
    print("=" * 80)
    
    # Load the dataset
    df = pd.read_csv(dataset_file, sep=';', decimal=',')
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Remove specified columns if present
    to_remove = [col for col in drop_cols if col in df.columns]
    if to_remove:
        df_clean = df.drop(columns=to_remove)
        print(f"Removed columns: {to_remove}")
    else:
        df_clean = df.copy()
        print(f"No columns to remove from {drop_cols}")
    
    # Parse date and sort
    df_clean['date'] = pd.to_datetime(df_clean['date'])
    df_clean = df_clean.sort_values('date').reset_index(drop=True)
    
    # Separate features and target
    X = df_clean.drop(columns=['date', target_col])
    y = df_clean[target_col]
    
    # Define feature types for proper preprocessing
    boolean_cols = ['holiday'] if 'holiday' in X.columns else []
    categorical_cols = ['weathercode'] if 'weathercode' in X.columns else []
    cyclical_cols = ['day_of_week', 'day_of_year', 'winddirection_10m_dominant'] if has_cyclical else []
    
    # All other columns are continuous and should be scaled
    continuous_cols = [col for col in X.columns 
                       if col not in boolean_cols + categorical_cols + cyclical_cols]
    
    # Convert continuous columns to numeric
    for col in continuous_cols:
        X[col] = pd.to_numeric(X[col], errors='coerce')
    y = pd.to_numeric(y, errors='coerce')
    
    print(f"\nFeatures shape: {X.shape}")
    print(f"Target shape: {y.shape}")
    
    # Apply cyclical encoding if needed
    def cyclical_encode(df, col, max_val):
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
        return df
    
    X_encoded = X.copy()
    
    if has_cyclical:
        if 'day_of_week' in X_encoded.columns:
            X_encoded = cyclical_encode(X_encoded, 'day_of_week', 7)
        if 'winddirection_10m_dominant' in X_encoded.columns:
            X_encoded = cyclical_encode(X_encoded, 'winddirection_10m_dominant', 360)
        if 'day_of_year' in X_encoded.columns:
            X_encoded = cyclical_encode(X_encoded, 'day_of_year', 366)
        # Drop original cyclical columns
        cols_to_drop = [c for c in cyclical_cols if c in X_encoded.columns]
        X_encoded = X_encoded.drop(columns=cols_to_drop)
        print(f"Applied cyclical encoding, dropped: {cols_to_drop}")
    
    print(f"Shape after encoding: {X_encoded.shape}")
    
    # Update continuous_cols after cyclical encoding
    continuous_cols = [col for col in X_encoded.columns 
                       if col not in boolean_cols + categorical_cols]
    
    # Apply StandardScaler to continuous features
    scaler = StandardScaler()
    X_scaled = X_encoded.copy()
    if continuous_cols:
        X_scaled[continuous_cols] = scaler.fit_transform(X_encoded[continuous_cols])
    
    print(f"Scaled {len(continuous_cols)} continuous features")
    
    # Time Series Split
    tscv = TimeSeriesSplit(n_splits=10)
    
    # Define models
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'ElasticNet': ElasticNet(),
        'Decision Tree': DecisionTreeRegressor(random_state=42),
        'Random Forest': RandomForestRegressor(random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'AdaBoost': AdaBoostRegressor(random_state=42),
        'XGBoost': XGBRegressor(random_state=42, n_jobs=-1, verbosity=0),
        'LightGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
        'K-Nearest Neighbors': KNeighborsRegressor(),
        'SVR': SVR(C=1e6, epsilon=1e4, kernel='rbf'),
    }
    
    # Evaluate models function
    def evaluate_model(model, X, y, tscv):
        rmse_scores, mae_scores, r2_scores = [], [], []
        for train_idx, test_idx in tscv.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
            mae_scores.append(mean_absolute_error(y_test, y_pred))
            r2_scores.append(r2_score(y_test, y_pred))
        return {
            'RMSE_mean': np.mean(rmse_scores), 'RMSE_std': np.std(rmse_scores),
            'MAE_mean': np.mean(mae_scores), 'MAE_std': np.std(mae_scores),
            'R2_mean': np.mean(r2_scores), 'R2_std': np.std(r2_scores),
        }
    
    # Train and evaluate all models
    print("\nTraining and evaluating models...")
    results = {}
    for name, model in models.items():
        try:
            metrics = evaluate_model(clone(model), X_scaled, y, tscv)
            results[name] = metrics
            print(f"  {name}: R² = {metrics['R2_mean']:.4f}")
        except Exception as e:
            print(f"  {name}: Error - {str(e)}")
            results[name] = {'RMSE_mean': np.nan, 'MAE_mean': np.nan, 'R2_mean': np.nan}
    
    all_results[dataset_name] = results
    
    # Find best model
    results_df = pd.DataFrame(results).T
    results_df = results_df.sort_values('R2_mean', ascending=False)
    best_model = results_df['R2_mean'].idxmax()
    print(f"\nBest Model: {best_model} (R² = {results_df.loc[best_model, 'R2_mean']:.4f})")
    
    # Optimal Training Timespan Analysis
    print("\n--- Training Timespan Analysis ---")
    
    test_end_date = df_clean['date'].max()
    test_start_date = test_end_date - pd.DateOffset(years=1)
    test_mask = df_clean['date'] > test_start_date
    X_test_final = X_scaled[test_mask]
    y_test_final = y[test_mask]
    
    train_available_mask = df_clean['date'] <= test_start_date
    train_start_date = df_clean[train_available_mask]['date'].min()
    train_end_date = df_clean[train_available_mask]['date'].max()
    total_train_years = (train_end_date - train_start_date).days / 365.25
    max_years = int(total_train_years)
    
    print(f"Test period: {test_start_date.date()} to {test_end_date.date()} ({len(X_test_final)} samples)")
    print(f"Max training years available: {max_years}")
    
    # Top 3 models for timespan analysis
    top_models = {
        'Gradient Boosting': GradientBoostingRegressor(random_state=42),
        'LightGBM': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
        'Random Forest': RandomForestRegressor(random_state=42),
    }
    
    timespan_results = {name: {'years': [], 'rmse': [], 'mae': [], 'r2': []} 
                        for name in top_models.keys()}
    
    for n_years in range(1, max_years + 1):
        train_period_start = test_start_date - pd.DateOffset(years=n_years)
        train_mask = (df_clean['date'] > train_period_start) & (df_clean['date'] <= test_start_date)
        X_train = X_scaled[train_mask]
        y_train = y[train_mask]
        
        for model_name, model in top_models.items():
            model_clone = clone(model)
            model_clone.fit(X_train, y_train)
            y_pred = model_clone.predict(X_test_final)
            rmse = np.sqrt(mean_squared_error(y_test_final, y_pred))
            mae = mean_absolute_error(y_test_final, y_pred)
            r2 = r2_score(y_test_final, y_pred)
            timespan_results[model_name]['years'].append(n_years)
            timespan_results[model_name]['rmse'].append(rmse)
            timespan_results[model_name]['mae'].append(mae)
            timespan_results[model_name]['r2'].append(r2)
    
    all_timespan_results[dataset_name] = timespan_results
    
    # Find optimal timespan
    timespan_df_list = []
    for model_name, res in timespan_results.items():
        for i in range(len(res['years'])):
            timespan_df_list.append({
                'Model': model_name, 'Training Years': res['years'][i],
                'RMSE': res['rmse'][i], 'MAE': res['mae'][i], 'R²': res['r2'][i]
            })
    timespan_df = pd.DataFrame(timespan_df_list)
    
    avg_r2_by_years = timespan_df.groupby('Training Years')['R²'].mean()
    optimal_years_overall = avg_r2_by_years.idxmax()
    optimal_r2_overall = avg_r2_by_years.max()
    
    print(f"\nOptimal training timespan: {optimal_years_overall} year(s) (avg R² = {optimal_r2_overall:.4f})")
    
    # Export data with optimal training timespan
    optimal_train_start = test_start_date - pd.DateOffset(years=optimal_years_overall)
    export_mask = df_clean['date'] > optimal_train_start
    
    export_df_optimal = X_scaled[export_mask].copy()
    export_df_optimal['slp'] = y[export_mask].values
    export_df_optimal['date'] = df_clean.loc[export_mask, 'date'].values
    
    cols = ['date', 'slp'] + [col for col in export_df_optimal.columns if col not in ['date', 'slp']]
    export_df_optimal = export_df_optimal[cols]
    
    export_df_optimal.to_csv(output_file, sep=';', decimal=',', index=False)
    print(f"\nExported to '{output_file}' ({len(export_df_optimal)} samples)")

print("\n" + "=" * 80)
print("ALL DATASETS PROCESSED SUCCESSFULLY!")
print("=" * 80)


PROCESSING DATASET: STAT
Input: dataset/df_clean_stat.csv

Dataset shape: (3560, 13)
Columns: ['date', 'temperature_2m_mean', 'sunrise', 'et0_fao_evapotranspiration', 'sunshine_duration', 'snowfall_sum', 'day_of_year', 'precipitation_hours', 'weathercode', 'windspeed_10m_max', 'rain_sum', 'holiday', 'Residential (SLP)']
No columns to remove from ['rlm', 'entry']

Features shape: (3560, 11)
Target shape: (3560,)
Shape after encoding: (3560, 11)
Scaled 9 continuous features

Training and evaluating models...
  Linear Regression: R² = 0.9116
  Ridge Regression: R² = 0.9115
  Lasso Regression: R² = 0.9116
  ElasticNet: R² = 0.8508
  Decision Tree: R² = 0.9076
  Random Forest: R² = 0.9405
  Gradient Boosting: R² = 0.9421
  AdaBoost: R² = 0.9061
  XGBoost: R² = 0.9377
  LightGBM: R² = 0.9397
  K-Nearest Neighbors: R² = 0.8858
  SVR: R² = 0.8551

Best Model: Gradient Boosting (R² = 0.9421)

--- Training Timespan Analysis ---
Test period: 2024-09-30 to 2025-09-30 (364 samples)
Max training ye

## 2. Results Summary

Display model performance and optimal training timespan for all datasets.


In [30]:
# Summary of all results
print("=" * 80)
print("COMPREHENSIVE RESULTS SUMMARY - ALL DATASETS")
print("=" * 80)

for dataset_name, results in all_results.items():
    print(f"\n{'─' * 40}")
    print(f"Dataset: {dataset_name.upper()}")
    print(f"{'─' * 40}")
    
    results_df = pd.DataFrame(results).T
    results_df = results_df.sort_values('R2_mean', ascending=False)
    
    # Format for display
    results_display = results_df.copy()
    results_display['RMSE'] = results_display.apply(lambda x: f"{x['RMSE_mean']:.2f} ± {x['RMSE_std']:.2f}", axis=1)
    results_display['MAE'] = results_display.apply(lambda x: f"{x['MAE_mean']:.2f} ± {x['MAE_std']:.2f}", axis=1)
    results_display['R²'] = results_display.apply(lambda x: f"{x['R2_mean']:.4f} ± {x['R2_std']:.4f}", axis=1)
    
    print("\nTop 5 Models:")
    print(results_display[['RMSE', 'MAE', 'R²']].head().to_string())


COMPREHENSIVE RESULTS SUMMARY - ALL DATASETS

────────────────────────────────────────
Dataset: STAT
────────────────────────────────────────

Top 5 Models:
                                   RMSE                   MAE               R²
Gradient Boosting  152490.86 ± 53817.95   95596.49 ± 21410.88  0.9421 ± 0.0547
Random Forest      154981.07 ± 53730.07   96681.97 ± 24726.36  0.9405 ± 0.0541
LightGBM           157027.86 ± 52364.82   98323.66 ± 22699.18  0.9397 ± 0.0549
XGBoost            160045.79 ± 52431.26   99812.10 ± 23490.64  0.9377 ± 0.0542
Lasso Regression   197656.30 ± 48214.17  146446.35 ± 28582.40  0.9116 ± 0.0645

────────────────────────────────────────
Dataset: RF
────────────────────────────────────────

Top 5 Models:
                                     RMSE                  MAE               R²
SVR                  146579.26 ± 56414.68  91534.67 ± 26548.84  0.9447 ± 0.0551
Gradient Boosting    151414.14 ± 52744.10  95270.30 ± 22528.45  0.9431 ± 0.0536
Random Forest      

In [31]:
# Timespan Analysis Summary
print("=" * 80)
print("OPTIMAL TRAINING TIMESPAN - ALL DATASETS")
print("=" * 80)

for dataset_name, timespan_results in all_timespan_results.items():
    print(f"\n{'─' * 40}")
    print(f"Dataset: {dataset_name.upper()}")
    print(f"{'─' * 40}")
    
    # Build dataframe
    timespan_df_list = []
    for model_name, res in timespan_results.items():
        for i in range(len(res['years'])):
            timespan_df_list.append({
                'Model': model_name, 'Years': res['years'][i],
                'RMSE': res['rmse'][i], 'R²': res['r2'][i]
            })
    timespan_df = pd.DataFrame(timespan_df_list)
    
    # Find optimal for each model
    for model_name in timespan_results.keys():
        model_data = timespan_df[timespan_df['Model'] == model_name]
        best_idx = model_data['R²'].idxmax()
        best_row = timespan_df.loc[best_idx]
        print(f"  {model_name}: {int(best_row['Years'])} year(s) → R² = {best_row['R²']:.4f}")
    
    # Overall recommendation
    avg_r2 = timespan_df.groupby('Years')['R²'].mean()
    optimal_years = avg_r2.idxmax()
    print(f"  >> Overall recommendation: {optimal_years} year(s)")

print("\n" + "=" * 80)
print("All output files have been saved to the dataset/ folder")
print("=" * 80)


OPTIMAL TRAINING TIMESPAN - ALL DATASETS

────────────────────────────────────────
Dataset: STAT
────────────────────────────────────────
  Gradient Boosting: 2 year(s) → R² = 0.9747
  LightGBM: 2 year(s) → R² = 0.9731
  Random Forest: 2 year(s) → R² = 0.9781
  >> Overall recommendation: 2 year(s)

────────────────────────────────────────
Dataset: RF
────────────────────────────────────────
  Gradient Boosting: 3 year(s) → R² = 0.9771
  LightGBM: 2 year(s) → R² = 0.9747
  Random Forest: 3 year(s) → R² = 0.9765
  >> Overall recommendation: 2 year(s)

────────────────────────────────────────
Dataset: FULL
────────────────────────────────────────
  Gradient Boosting: 3 year(s) → R² = 0.9763
  LightGBM: 2 year(s) → R² = 0.9749
  Random Forest: 2 year(s) → R² = 0.9748
  >> Overall recommendation: 3 year(s)

All output files have been saved to the dataset/ folder
